<a href="https://colab.research.google.com/github/chinmayee95/pixelhop-/blob/master/pixelhop%2B%2B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%cd "/content/"

/content


In [4]:
!git clone https://github.com/USC-MCL/EE569_2020Spring.git

fatal: destination path 'EE569_2020Spring' already exists and is not an empty directory.


In [5]:
!pwd
!ls EE569_2020Spring/

/content
cross_entropy.py  lag.py   pixelhop2.pkl  __pycache__  requirements.txt
cwSaab.py	  llsr.py  pixelhop2.py   README.md    saab.py


In [6]:
!pwd

/content


In [7]:
%cd /content/EE569_2020Spring/

/content/EE569_2020Spring


In [0]:
import numpy as np
import time
from cross_entropy import Cross_Entropy
from lag import LAG
from llsr import LLSR as myLLSR
from pixelhop2 import Pixelhop2
import skimage
from skimage.util import view_as_windows
import skimage.measure
import numpy as np

In [0]:
def Concat(X, concatArg):
    return X

In [0]:
def Shrink(X, shrinkArg):
    # print(type(shrinkArg))

    # win = shrinkArg['win']
    win = 5
    hop1 = shrinkArg['hop']
    # stride = shrinkArg['stride']

    # lasthop = shrinkArgs['hoplast']
    lasthop = False
    stride = 1
    ch = X.shape[-1]
    print('input shape:',X.shape)
    if lasthop:
      # X = view_as_windows(X, (1,win,win,1), (1,win,win,1))
      X = X.reshape(X.shape[0], X.shape[1], X.shape[2], -1)
    else:
      if ~hop1:
        # win1=2
        X = skimage.measure.block_reduce(X, (1,2,2,1), np.max)
      X = view_as_windows(X, (1,win,win,ch), (1,stride,stride,ch))
      X = X.reshape(X.shape[0], X.shape[1], X.shape[2], -1)
      # if hop1:
        # X = skimage.measure.block_reduce(X, (1,win,win,1), np.max)
      print('output shape:',X.shape)
    return X  

In [0]:
# from skimage.util import view_as_windows
# def Shrink(X, shrinkArg):

#   win = shrinkArg['win']
#   win=5
#   hop1 = shrinkArg['hop']
#   # print(shrinkArg['hop'])
#   stride = 1
#   ch = X.shape[-1]
#   # if ~hop1:
#   #   X = skimage.measure.block_reduce(X, (1,2,2,1), np.max)
#     # keras.layers.MaxPooling2D(pool_size=(2, 2), strides=1, padding='same', data_format=None)
#   X = view_as_windows(X, (1,win,win,ch), (1,stride,stride,ch))
#   X = X.reshape(X.shape[0], X.shape[1], X.shape[2], -1)
#   # print(X.shape)
#   # if hop1:
#   #   X = skimage.measure.block_reduce(X, (1,2,2,1), np.max)
#   return X

In [0]:
from skimage.util import view_as_windows
def Shrink(X, shrinkArg):

  win = shrinkArg['win']
  win=5
  hop = shrinkArg['hop']
  print('Hop:',hop)
  stride = 1
  ch = X.shape[-1]
  print('Input shape:',X.shape)
  if hop==1:
    X = view_as_windows(X, (1,win,win,ch), (1,stride,stride,ch))
    X = X.reshape(X.shape[0], X.shape[1], X.shape[2], -1)
    X = skimage.measure.block_reduce(X, (1,2,2,1), np.max)
  if hop==2:
    X = view_as_windows(X, (1,win,win,ch), (1,stride,stride,ch))
    X = X.reshape(X.shape[0], X.shape[1], X.shape[2], -1)
    X = skimage.measure.block_reduce(X, (1,2,2,1), np.max)
  if hop==3:
    X = view_as_windows(X, (1,win,win,ch), (1,stride,stride,ch))
    X = X.reshape(X.shape[0], X.shape[1], X.shape[2], -1)
  print(X.shape)
  return X

In [0]:

"""neighborhood construction
shrink saab concat args
"""
SaabArgs = [{'num_AC_kernels':-1, 'needBias':False, 'useDC':True, 'batch':None, 'cw': True},
                {'num_AC_kernels':-1, 'needBias':True, 'useDC':True, 'batch':None, 'cw': True},
            {'num_AC_kernels':-1, 'needBias':True, 'useDC':True, 'batch':None, 'cw': True}]
shrinkArgs = [{'func':Shrink, 'win':2, 'hop':1},
              {'func': Shrink, 'win':2,'hop':2},
              {'func': Shrink, 'win':2, 'hop':3}]
concatArg = {'func':Concat}

In [0]:
from keras.datasets import cifar10
#preprocessing of data 
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train_50k = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255

In [44]:
#selecting 10k images
x_train_10k = x_train_50k[0:1000]
print(x_train_10k.shape)

(1000, 32, 32, 3)


In [16]:
#train 
phops = Pixelhop2(depth=3, TH1=0.01, TH2=0.001, SaabArgs=SaabArgs, shrinkArgs=shrinkArgs, concatArg=concatArg)
phops.fit(x_train_10k)
output = phops.transform(x_train_10k)

pixelhop2 fit
Hop: 1
Input shape: (1000, 32, 32, 1)
(1000, 14, 14, 25)
Hop: 1
Input shape: (1000, 32, 32, 1)
(1000, 14, 14, 25)
Hop: 1
Input shape: (1000, 32, 32, 1)
(1000, 14, 14, 25)
Hop: 2
Input shape: (1000, 14, 14, 1)
(1000, 5, 5, 25)
Hop: 2
Input shape: (1000, 14, 14, 1)
(1000, 5, 5, 25)
Hop: 2
Input shape: (1000, 14, 14, 1)
(1000, 5, 5, 25)
Hop: 2
Input shape: (1000, 14, 14, 1)
(1000, 5, 5, 25)
Hop: 2
Input shape: (1000, 14, 14, 1)
(1000, 5, 5, 25)
Hop: 2
Input shape: (1000, 14, 14, 1)
(1000, 5, 5, 25)
Hop: 2
Input shape: (1000, 14, 14, 1)
(1000, 5, 5, 25)
Hop: 2
Input shape: (1000, 14, 14, 1)
(1000, 5, 5, 25)
Hop: 2
Input shape: (1000, 14, 14, 1)
(1000, 5, 5, 25)
Hop: 2
Input shape: (1000, 14, 14, 1)
(1000, 5, 5, 25)
Hop: 2
Input shape: (1000, 14, 14, 1)
(1000, 5, 5, 25)
Hop: 2
Input shape: (1000, 14, 14, 1)
(1000, 5, 5, 25)
Hop: 2
Input shape: (1000, 14, 14, 1)
(1000, 5, 5, 25)
Hop: 2
Input shape: (1000, 14, 14, 1)
(1000, 5, 5, 25)
Hop: 2
Input shape: (1000, 14, 14, 1)
(1000, 

In [17]:
  print(output[0].shape)
  print(output[1].shape)
  print(output[2].shape)

(1000, 14, 14, 45)
(1000, 5, 5, 147)
(1000, 1, 1, 219)


In [18]:
#save model
#train 
phops = Pixelhop2(depth=3, TH1=0.01, TH2=0.001, SaabArgs=SaabArgs, shrinkArgs=shrinkArgs, concatArg=concatArg)
phops.fit(x_train_10k)

import pickle

# save
with open('pixelhop2.pkl','wb') as f:
    pickle.dump(phops,f)

# load
with open('pixelhop2.pkl', 'rb') as f:
    clf2 = pickle.load(f)


pixelhop2 fit
Hop: 1
Input shape: (1000, 32, 32, 1)
(1000, 14, 14, 25)
Hop: 1
Input shape: (1000, 32, 32, 1)
(1000, 14, 14, 25)
Hop: 1
Input shape: (1000, 32, 32, 1)
(1000, 14, 14, 25)
Hop: 2
Input shape: (1000, 14, 14, 1)
(1000, 5, 5, 25)
Hop: 2
Input shape: (1000, 14, 14, 1)
(1000, 5, 5, 25)
Hop: 2
Input shape: (1000, 14, 14, 1)
(1000, 5, 5, 25)
Hop: 2
Input shape: (1000, 14, 14, 1)
(1000, 5, 5, 25)
Hop: 2
Input shape: (1000, 14, 14, 1)
(1000, 5, 5, 25)
Hop: 2
Input shape: (1000, 14, 14, 1)
(1000, 5, 5, 25)
Hop: 2
Input shape: (1000, 14, 14, 1)
(1000, 5, 5, 25)
Hop: 2
Input shape: (1000, 14, 14, 1)
(1000, 5, 5, 25)
Hop: 2
Input shape: (1000, 14, 14, 1)
(1000, 5, 5, 25)
Hop: 2
Input shape: (1000, 14, 14, 1)
(1000, 5, 5, 25)
Hop: 2
Input shape: (1000, 14, 14, 1)
(1000, 5, 5, 25)
Hop: 2
Input shape: (1000, 14, 14, 1)
(1000, 5, 5, 25)
Hop: 2
Input shape: (1000, 14, 14, 1)
(1000, 5, 5, 25)
Hop: 2
Input shape: (1000, 14, 14, 1)
(1000, 5, 5, 25)
Hop: 2
Input shape: (1000, 14, 14, 1)
(1000, 

In [19]:
#choosing 10k images
x_train_50k
y_train

array([[6],
       [9],
       [9],
       ...,
       [9],
       [1],
       [1]], dtype=uint8)

In [20]:
#all train data - 50k images
#save model
#train 
phops = Pixelhop2(depth=3, TH1=0.01, TH2=0.001, SaabArgs=SaabArgs, shrinkArgs=shrinkArgs, concatArg=concatArg)
phops.fit(x_train_50k[0:10000])
output = phops.transform(x_train_50k[0:10000])

pixelhop2 fit
Hop: 1
Input shape: (10000, 32, 32, 1)
(10000, 14, 14, 25)
Hop: 1
Input shape: (10000, 32, 32, 1)
(10000, 14, 14, 25)
Hop: 1
Input shape: (10000, 32, 32, 1)
(10000, 14, 14, 25)
Hop: 2
Input shape: (10000, 14, 14, 1)
(10000, 5, 5, 25)
Hop: 2
Input shape: (10000, 14, 14, 1)
(10000, 5, 5, 25)
Hop: 2
Input shape: (10000, 14, 14, 1)
(10000, 5, 5, 25)
Hop: 2
Input shape: (10000, 14, 14, 1)
(10000, 5, 5, 25)
Hop: 2
Input shape: (10000, 14, 14, 1)
(10000, 5, 5, 25)
Hop: 2
Input shape: (10000, 14, 14, 1)
(10000, 5, 5, 25)
Hop: 2
Input shape: (10000, 14, 14, 1)
(10000, 5, 5, 25)
Hop: 2
Input shape: (10000, 14, 14, 1)
(10000, 5, 5, 25)
Hop: 2
Input shape: (10000, 14, 14, 1)
(10000, 5, 5, 25)
Hop: 2
Input shape: (10000, 14, 14, 1)
(10000, 5, 5, 25)
Hop: 2
Input shape: (10000, 14, 14, 1)
(10000, 5, 5, 25)
Hop: 2
Input shape: (10000, 14, 14, 1)
(10000, 5, 5, 25)
Hop: 2
Input shape: (10000, 14, 14, 1)
(10000, 5, 5, 25)
Hop: 2
Input shape: (10000, 14, 14, 1)
(10000, 5, 5, 25)
Hop: 2
Inpu

In [0]:
def cal_cross_entropy(features,Ns,y_train):
  ce = Cross_Entropy(num_class=10, num_bin=5)
  features = features.reshape((features.shape[0], -1))
  print(features.shape)
  # feat_ce = np.zeros((features.shape[0],features.shape[-1]))
  feat_ce = np.zeros(features.shape[-1])
  # for l in range(features.shape[0]):
  for k in range(features.shape[-1]):
    feat_ce[k] = ce.KMeans_Cross_Entropy(features[:,k].reshape(-1,1), y_train)
      # print(" --> KMeans ce: %s"%str(feat_ce[k]))
  print("------- DONE -------\n")
  # print(feat_ce.shape)
  # ce_sorted = feat_ce.sort()
  # ce_ns = ce_sorted[0:Ns]
  return feat_ce

In [22]:
print(output[0].shape[-1])
k = output[0].reshape((output[0].shape[0], -1))
# print(type(int(k)))
print(k.shape)

45
(10000, 8820)


In [24]:
ce_hop_unit1 = cal_cross_entropy(output[0],4000,y_train[0:10000])
ce_hop_unit2 = cal_cross_entropy(output[1],4000,y_train[0:10000])
ce_hop_unit3 = cal_cross_entropy(output[2],4000,y_train[0:10000])

(10000, 8820)
------- DONE -------

(10000, 3525)
------- DONE -------

(10000, 218)
------- DONE -------



In [0]:
a = np.asarray([[1,2],[3,4],[2,2]])
b = np.asarray([[1,2],[4,4],[2,2]])
c = np.argwhere(a==b)
print('hi',c)
print(a.shape)
a = np.mean(a, axis=0)
# print(a)

In [0]:
def crossentropy_sort(crossentropy,Ns):
  # print(type(crossentropy))
  # print(crossentropy)

  indices = crossentropy.argsort()[:1000]
  # print("Indices", indices)
  # crossentropy = np.expand_dims(crossentropy,axis=1)
  # print(crossentropy.shape)
  # ce_avg = np.mean(crossentropy,axis=0)
  # print(ce_avg)
  # ce_sorted = sorted(crossentropy)
  # print("SORTED VALUES",ce_sorted)
  # print("LENGTH", len(ce_sorted))
  # temp_list =  list(ce_sorted)
  # ce_sorted = ce_sorted[0]
  # ce_sorted = np.expand_dims(ce_sorted,axis=1)
  # print(ce_sorted.shape)
  # ce_final = np.zeros(crossentropy.shape)
  # indx = 1
  # print(indx)
  # indx = np.zeros(Ns)
  # for i in range(Ns):
    # ce_final[:,i] = ce_avg(indx[i])
  #reduce dimensions selecting Ns dimensions
  # print(len(ce_sorted))
  # ce_ns = ce_sorted[0:Ns]
  # indx = indx[0:Ns]
  # print(len(indx))
  return indices

In [0]:
Ns = 1000
ce_sorted_unit1 = crossentropy_sort(ce_hop_unit1,Ns)
ce_sorted_unit2 = crossentropy_sort(ce_hop_unit2,Ns)
ce_sorted_unit3 = crossentropy_sort(ce_hop_unit3,Ns)
# print(len(ce_sorted_unit3))

In [0]:
def feature_selection(data,index_array,Ns):
  # print(data.shape[-1])
  out = data.reshape((data.shape[0], -1))
  print(out.shape)
  # print(type(int(k)))
  mini = min(Ns,out.shape[-1])
  result = np.zeros((out.shape[0],mini))
  print(out.shape)
  print(result.shape)
  j=1
  for i in index_array:
    if j<mini:
      result[:,j] = out[:,i]
      j = j + 1
    if j==1000:
      exit
  return result



In [54]:
#feature selection - Ns
# print(output[0].shape)
# print(output[0].shape[-1])
# out = output[0].reshape((output[0].shape[0], -1))
# print(out.shape[0])
Ns = 1000
fs_hop_unit1 = feature_selection(output[0],ce_sorted_unit1,Ns)
fs_hop_unit2 = feature_selection(output[1],ce_sorted_unit2,Ns)
fs_hop_unit3 = feature_selection(output[2],ce_sorted_unit3,Ns)
# result = np.zeros((out.shape[0],1000))
# print(result.shape)

(10000, 8820)
(10000, 8820)
(10000, 1000)
(10000, 3525)
(10000, 3525)
(10000, 1000)
(10000, 218)
(10000, 218)
(10000, 218)


In [56]:
# fs_hop_unit1 = np.expand_dims(fs_hop_unit1,axis=1)
fs_hop_unit3.shape

(10000, 218)

In [0]:
def lag_compute(x_train, y_train,x_test, y_test,alpha=10):
    # x_train = np.asarray(x_train)
    # x_train = np.expand_dims(x_train, axis=1)
    print(x_train.shape)
    lag = LAG(encode='distance', num_clusters=[2,2,2,2,2,2,2,2,2,2], alpha=10, learner=myLLSR(onehot=False))  
    lag.fit(x_train, y_train)
    x_train_trans = lag.transform(x_train)
    x_test_trans = lag.transform(x_test)
    x_train_predprob = lag.predict_proba(x_train)
    print('test size:', x_test.shape)
    print(" --> train acc: %s"%str(lag.score(x_train, y_train)))
    print(" --> test acc.: %s"%str(lag.score(x_test, y_test)))
    print("------- DONE -------\n")
    return x_train_trans,x_test_trans

In [0]:
def resize_2d(data):
  out = data.reshape((data.shape[0], -1))
  return out

In [67]:
x_test = resize_2d(x_test)
x_test_fs = feature_selection(x_test,ce_sorted_unit3,Ns)

(10000, 3072)
(10000, 3072)
(10000, 1000)


In [68]:
alpha = 10
y_train = y_train[0:10000]
lag_unit1 = lag_compute(fs_hop_unit1,y_train,x_test_fs, y_test,alpha)
lag_unit2 = lag_compute(fs_hop_unit2,y_train,x_test_fs, y_test,alpha)
lag_unit3 = lag_compute(fs_hop_unit3,y_train,x_test_fs[:,0:218], y_test,alpha)

(10000, 1000)
test size: (10000, 1000)
 --> train acc: 0.5455
 --> test acc.: 0.109
------- DONE -------

(10000, 1000)
test size: (10000, 1000)
 --> train acc: 0.6709
 --> test acc.: 0.0955
------- DONE -------

(10000, 218)
test size: (10000, 218)
 --> train acc: 0.5075
 --> test acc.: 0.1061
------- DONE -------



In [74]:
#concatenate the outputs of lag
lags0 = np.concatenate((lag_unit1[0],lag_unit2[0],lag_unit3[0]),axis = 1)
lags1 = np.concatenate((lag_unit1[1],lag_unit2[1],lag_unit3[1]),axis = 1)

print(lags0.shape)

(10000, 60)


In [77]:
from llsr import LLSR
reg = LLSR(onehot=True, normalize=False)
reg.fit(lags0, y_train)
X_train_reg = reg.predict_proba(lags0)
print(" --> train acc: %s"%str(reg.score(lags0, y_train)))
print(" --> test acc: %s"%str(reg.score(lags1, y_test)))
print("------- DONE -------\n")

 --> train acc: 0.7113
 --> test acc: 0.1112
------- DONE -------



In [78]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
scaler=preprocessing.StandardScaler()
feature = scaler.fit_transform(lags0)
feature_test = scaler.transform(lags1)     
   
clf=SVC().fit(feature, y_train) 
##        clf=RandomForestClassifier(n_estimators=500,max_depth=5).fit(train_f, train_labels) 
print('***** Train ACC:', accuracy_score(y_train,clf.predict(feature)))
print('***** Test ACC:', accuracy_score(y_test,clf.predict(feature_test)))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


***** Train ACC: 0.852
***** Test ACC: 0.1


In [85]:
#classifier
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=20, random_state=0)
feature = scaler.fit_transform(lags0)
feature_test = scaler.transform(lags1)  
clf = clf.fit(feature, y_train)

# RandomForestClassifier(max_depth=2, random_state=0)
# print(clf.feature_importances_)
# print(clf.predict(lags1))
print('***** Train ACC:', accuracy_score(y_train,clf.predict(feature)))
print('***** Test ACC:', accuracy_score(y_test,clf.predict(feature_test)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


***** Train ACC: 0.9649
***** Test ACC: 0.0792


In [0]:
# Plot confusion matrix
from sklearn.metrics import confusion_matrix
import itertools
plt.rcParams['figure.figsize'] = [10,7]

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):

  if normalize:
      cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
      print("Normalized confusion matrix")
  else:
      print('Confusion matrix, without normalization')

  print(cm)

  plt.imshow(cm, interpolation='nearest', cmap=cmap)
  plt.title(title)
  plt.colorbar()
  tick_marks = np.arange(len(classes))
  plt.xticks(tick_marks, classes, rotation=45)
  plt.yticks(tick_marks, classes)

  fmt = '.2f' if normalize else 'd'
  thresh = cm.max() / 2.
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
      plt.text(j, i, format(cm[i, j], fmt),
               horizontalalignment="center",
               color="white" if cm[i, j] > thresh else "black")

  plt.tight_layout()
  plt.ylabel('True label')
  plt.xlabel('Predicted label')
  plt.show()


p_test = clf.predict(x_test).argmax(axis=1)
cm = confusion_matrix(y_test, p_test)
plot_confusion_matrix(cm, list(range(10)))

In [0]:
import numpy as np
a = np.ones((3,4,5))
b = np.ones((3,4,5))
c = np.concatenate((a[...,np.newaxis],b[...,np.newaxis]),axis=3)